In [ ]:
!unzip /content/max-indabax-nigeria-hackathon.zip

Archive:  /content/max-indabax-nigeria-hackathon.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer,IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train=pd.read_csv("/content/train.csv",index_col=0)
test=pd.read_csv("/content/test.csv",index_col=0)

In [ ]:
ind=test.index

In [ ]:
target=train["label"]
train.drop("label", axis=1, inplace=True)

In [ ]:
tar=list(target.value_counts().index)

In [ ]:
data=pd.concat([train,test])

In [ ]:
hashmap={}
list_counts=[]
for i in range(data.shape[0]):
  date=data["date_time"].iloc[i]
  agent=data["subscriber_id"].iloc[i]
  if (date,agent) in hashmap:
    list_counts.append(hashmap[(date,agent)]+1)
    hashmap[(date,agent)]=hashmap[(date,agent)]+1
  else:
    list_counts.append(0)
    hashmap[(date,agent)]=0

In [ ]:
data["no_occurence_subs"]=list_counts

In [ ]:
data["date_time_x"]=pd.to_datetime(data["date_time"])
data["date_time_year"]=data["date_time_x"].dt.year
data["date_time_day"]=data["date_time_x"].dt.day
data["date_time_mnth"]=data["date_time_x"].dt.month
data["agent_day"]=data["date_time_day"].astype("str")+data["date_time_mnth"].astype("str")+data["date_time_year"].astype("str")+data["agent_id"]
data.drop(["date_time_x","date_time_day"],axis=1,inplace=True)

In [ ]:
data["agent_day"]=data["agent_day"].map(data["agent_day"].value_counts().to_dict())

In [ ]:
cat_columns=[]
for i in data.columns:
  if data[i].dtype=="object":
    data[i]=pd.factorize(data[i])[0]
    cat_columns.append(i)

In [ ]:
data=data.fillna(method="bfill")

In [ ]:
train=data.iloc[:train.shape[0]]
test=data.iloc[train.shape[0]:]

In [ ]:
X,val,y,y_val=train_test_split(train, target, test_size=0.2,random_state=0)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model=CatBoostClassifier(random_state=0,use_best_model=True,verbose=50,n_estimators=1000,early_stopping_rounds=50,cat_features=cat_columns)#,auto_class_weights="Balanced")

In [ ]:
model.fit(X,y,eval_set=(val,y_val))

Learning rate set to 0.5
0:	learn: 1.7981972	test: 1.7922142	best: 1.7922142 (0)	total: 5.08s	remaining: 0us

bestTest = 1.792214185
bestIteration = 0



In [ ]:
sub=pd.DataFrame(model.predict_proba(test))

In [ ]:
sub.columns=model.classes_

In [ ]:
sub["ID"]=ind

In [ ]:
sub.set_index("ID", inplace=True)

In [ ]:
sub.columns

Index(['Activated', 'Awaiting Activation', 'Awaiting Vehicle Pickup',
       'Backlog', 'Checked In For Test', 'In Verification',
       'Issued Guarantor Form', 'Onboarding', 'Received Guarantor Form',
       'Test Scheduled', 'Tested', 'Top of the funnel'],
      dtype='object')

In [ ]:
log_loss(y_val,model.predict_proba(val),labels=tar)

1.7922141850472624

In [ ]:
log_loss(y_val,model.predict_proba(val),labels=tar)

1.7922141850472624

In [ ]:
sub

,Activated,Awaiting Activation,Awaiting Vehicle Pickup,Backlog,Checked In For Test,In Verification,Issued Guarantor Form,Onboarding,Received Guarantor Form,Test Scheduled,Tested,Top of the funnel
ID,,,,,,,,,,,,
ID_04I3F51N,0.078807,0.058626,0.112716,0.048874,0.107989,0.067705,0.094984,0.067188,0.066380,0.105947,0.082651,0.108133
ID_RL7OZ03G,0.078807,0.058626,0.112716,0.048874,0.107989,0.067705,0.094984,0.067188,0.066380,0.105947,0.082651,0.108133
ID_880UB1KT,0.078807,0.058626,0.112716,0.048874,0.107989,0.067705,0.094984,0.067188,0.066380,0.105947,0.082651,0.108133
ID_P46W8LVS,0.039537,0.035844,0.038924,0.034214,0.062397,0.037105,0.046020,0.037055,0.037933,0.250694,0.043371,0.336907
ID_JHON927V,0.039537,0.035844,0.038924,0.034214,0.062397,0.037105,0.046020,0.037055,0.037933,0.250694,0.043371,0.336907
...,...,...,...,...,...,...,...,...,...,...,...,...
ID_Y52L9ECX,0.078807,0.058626,0.112716,0.048874,0.107989,0.067705,0.094984,0.067188,0.066380,0.105947,0.082651,0.108133
ID_HW1J30MC,0.039537,0.035844,0.038924,0.034214,0.062397,0.037105,0.046020,0.037055,0.037933,0.250694,0.043371,0.336907
ID_JUCRN3HX,0.032157,0.032157,0.032220,0.032157,0.032157,0.032157,0.193114,0.032157,0.211120,0.306286,0.032157,0.032157


In [ ]:
sub.to_csv("max1.csv")